In [1]:
import sys
sys.path.insert(0, r'C:\Users\buzga\Desktop\School\Reaserch\Langone\ML_RA_EHR\DataModule')
sys.path.insert(0, r'C:\Users\buzga\Desktop\School\Reaserch\Langone\ML_RA_EHR')
sys.path.insert(0, r'C:\Users\buzga\Desktop\School\Reaserch\Langone\ML_RA_EHR\EnsambleModule')
import Data_Preparation
import EvaluationModule 
import ensamble_model
from pandas.api.types import is_dict_like
import sklearn as sk
from sklearn import neural_network
from sklearn import naive_bayes
from sklearn import ensemble
from sklearn import tree 
import xgboost as xgb


In [6]:
## building stacking modle for regression
model_list={"Linear":{} ,'ANN':{}, 'Ridge':{}, 'Lasso':{}, 'SVM':{},'Random Forest':{}, "Tree":{}, "XGBoost":{} }
challange='Regression'
regression_models=ensamble_model.make_ensamble(model_list=model_list, challange=challange).models
regression_models["Baggign_Regression_SVM"]=ensamble_model.make_ensamble(model_list={"SVM":{}}, challange=challange, ensamble_type="Bagging").model
regression_models["Baggign_Regression_SVM"]=ensamble_model.make_ensamble(model_list={"SVM":{}}, challange=challange, ensamble_type="Boosting").model
regression_models

{'Linear': LinearRegression(),
 'ANN': MLPRegressor(),
 'Ridge': Ridge(),
 'Lasso': Lasso(),
 'SVM': SVR(),
 'Random Forest': RandomForestRegressor(),
 'Tree': DecisionTreeRegressor(),
 'XGBoost': XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None,
              reg_alpha=None, reg_lambda=None, ...),
 'Ensamble': StackingRegressor(estimators=[('Linear', LinearRe

In [8]:
path=r'C:\Users\buzga\Desktop\School\Reaserch\Langone\ML_RA_EHR\Dataset'
dataset = Data_Preparation.CoronnaCERTAINDataset(
    library_root=path,
    challenge="regression", #option: regression, regression_delta, classification, binary_classification
    dataset='CORRONA CERTAIN', 
    process_approach='SC', #option: KVB, SC
    imputation="IterativeImputer", #option: SimpleFill, KNN, SoftImpute, BiScaler, NuclearNormMinimization, IterativeImputer, IterativeSVD, None(raw)
    patient_group='bionaive TNF', #option: "all", "bioexp nTNF", "bionaive TNF", "bionaive orencia", "KVB"
    drug_group='all', #option: "all", "actemra", "cimzia", "enbrel", "humira", "orencia", "remicade", "rituxan", "simponi"
    time_points=(0,3), 
    train_test_rate=0.8,
    remove_low_DAS = True,
    save_csv=True,
    random_state=2022)



train, train_loc = dataset.get_train()
test, test_loc = dataset.get_test()


## train test split 
X_train = train.iloc[:,:-1]
y_train = train.iloc[:,-1]
X_test = test.iloc[:,:-1]

y_test = test.iloc[:,-1]
len(y_test)+len(y_train)


feature engineering, drop columns due to 70% missing value: Index(['smkyrs', 'numcigs', 'rfstatus_impute', 'ccpstatus_impute',
       'statin_use'],
      dtype='object')
Missing values in train before imputation: 128
Missing values in train after imputation: 0
Missing values in test before imputation: 24
Missing values in test after imputation: 0
save file to: C:\Users\buzga\Desktop\School\Reaserch\Langone\ML_RA_EHR\Dataset\Coronna_Data_CERTAIN_regression_SC_0M_3M_bionaive TNF_all\Train.csv
save file to: C:\Users\buzga\Desktop\School\Reaserch\Langone\ML_RA_EHR\Dataset\Coronna_Data_CERTAIN_regression_SC_0M_3M_bionaive TNF_all\Test.csv


389

In [9]:
aml=EvaluationModule.AutoBuild(seed=1,challenge='regression')

for model in regression_models:
   aml.validate(model,regression_models[model],train,test)


c:\Users\buzga\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\buzga\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\buzga\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\buzga\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\buzga\anaconda3\lib\site-packages\sklearn\neural_network\_multi

In [ ]:
aml.validation_output(dataset=dataset)
aml.test_output(dataset=dataset)# use this for testing
